In [3]:
import datetime 

from sklearn import datasets

from evidently.metrics import ColumnDriftMetric, ColumnSummaryMetric
from evidently.metrics import DatasetDriftMetric, DatasetMissingValuesMetric
from evidently.report import Report 
from evidently.test_preset import DataDriftTestPreset
from evidently.test_suite import TestSuite
from evidently.ui.dashboards import CounterAgg, DashboardPanelCounter
from evidently.ui.dashboards import DashboardPanelPlot, PanelValue
from evidently.ui.dashboards import PlotType, ReportFilter
from evidently.ui.remote import RemoteWorkspace
from evidently.ui.workspace import Workspace, WorkspaceBase

In [4]:
adult_data = datasets.fetch_openml(name="adult", version=2, as_frame="auto")
adult = adult_data.frame

C:\Users\User\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
adult_ref = adult[~adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]
adult_cur = adult[adult.education.isin(["Some-college", "HS-grad", "Bachelors"])]

In [12]:
WORKSPACE = "Monitoring"

PROJECT_NAME = "Evidently Monitoring"
PROJECT_DESCRIPTION = "This is my first evidently monittoring hands on tutorial."

In [13]:
def create_report(i: int):
    data_drift_report = Report(
        metrics=[
            DatasetDriftMetric(),
            DatasetMissingValuesMetric(),
            ColumnDriftMetric(column_name="age", stattest="wasserstein"),
            ColumnSummaryMetric(column_name="age"),
            ColumnDriftMetric(column_name="education-num", stattest="wasserstein"),
            ColumnSummaryMetric(column_name="education-num"),
        ],
        timestamp=datetime.datetime.now() + datetime.timedelta(days=i),
    )
    
    data_drift_report.run(reference_data=adult_ref, current_data=adult_cur.iloc[100 * i : 100 * (i + 1), :])
    return data_drift_report

In [24]:
def create_project(workspace: WorkspaceBase):
    project = workspace.create_project(PROJECT_NAME)
    project.description = PROJECT_DESCRIPTION
    
    project.dashboard.add_panel(
        DashboardPanelCounter(
            title="Model Calls",
            filter=ReportFilter(metadata_values={}, tag_values=[]),
            value=PanelValue(
                metric_id="DatasetMissingValuesMetric",
                field_path=DatasetMissingValuesMetric.fields.current.number_of_rows,
                legend="count",
            ),
            text="count",
            agg=CounterAgg.SUM,
            size=2,
        )
    )
    project.save()
    return project

In [25]:
def create_monitoring_project(workspace: str):
    ws  =Workspace.create(workspace)
    project = create_project(ws)
    
    for i in range(0, 5):
        report = create_report(i=i)
        ws.add_report(project.id, report)
        
        #test_suite = create_test_suite(i=i)
        #ws.add_test_suite(project.id, test_suite)
        
if __name__=="__main__":
    create_monitoring_project("New Workspace")